# Dendrite Phase Field Benchmark

FiPy implementation of 2D dendritic solidification benchmark.

https://pages.nist.gov/pfhub/benchmarks/benchmark3.ipynb/

## Import Python modules

In [1]:
import json
import sys
from scipy.interpolate import CubicSpline
from scipy.optimize import root_scalar

import fipy as fp
from fipy import numerix as nmx

In [2]:
isnotebook = False
try:
    from IPython import get_ipython
    isnotebook = (get_ipython().__class__.__name__ == "ZMQInteractiveShell")
except:
    pass

## Define simulation parameters

In [2]:
sys.argv

['/Users/guyer/mambaforge-arm/envs/fipy311/lib/python3.11/site-packages/ipykernel_launcher.py',
 '-f',
 '/Users/guyer/Library/Jupyter/runtime/kernel-45a37b1f-3165-4a50-9bac-55dce0fbff41.json']

In [3]:
if isnotebook:
    params = {
        "dx": 0.1,
        "Lx": 96.,
        "r0": 8.,
        "dt": 0.5,
        "t_max": 100,
        "view": False
    }
    output = "dx=0_1dt=0_5.csv"
else:
    with open(sys.argv[1], "r") as f:
        params = json.load(f)
    output = sys.argv[2]

In [140]:
with open("params.json", "w") as f:
    json.dump(params, f)

## Define domain

In [4]:
mesh = fp.Grid1D(dx=params["dx"], Lx=params["Lx"])

## Define solution variables

In [5]:
u = fp.CellVariable(mesh=mesh, name="u", hasOld=True)

In [6]:
phi = fp.CellVariable(mesh=mesh, name=r"$\varphi$", hasOld=True)

## Define equations

In [7]:
D = 10.
tau0 = 1.
W0 = 1.
lamda = D * tau0 / (0.6267 * W0**2)
norm = phi.grad / (phi.grad.mag + (phi.grad.mag == 0.) * 1.)
theta = fp.Variable(0.)
theta.name = r"$\theta$"
m = fp.Variable(4)
epsilon = fp.Variable(0.05)
a = 1 + epsilon * nmx.cos(m * theta)
aPrime = -epsilon * nmx.sin(m * theta) * m
tau = tau0 * a**2
W = W0 * a
Wprime = W0 * aPrime
Dphi = W**2
Delta = fp.Variable(value=0.3, name=r"$\Delta$")

In [8]:
ueq = (fp.TransientTerm(coeff=1., var=u) 
       == fp.DiffusionTerm(coeff=D, var=u)
       + fp.TransientTerm(coeff=0.5, var=phi))

In [9]:
phieq = (fp.TransientTerm(coeff=tau, var=phi)
         == fp.DiffusionTerm(coeff=Dphi, var=phi)
         + fp.ImplicitSourceTerm(coeff=1-phi**2, var=phi)
         - fp.ImplicitSourceTerm(coeff=lamda * (1 - 2*phi**2 + phi**4), var=u))

In [10]:
eq = ueq & phieq

## Define boundary conditions

In [11]:
u.constrain(-Delta, where=mesh.facesRight)

## Define metrics

In [12]:
f_chem = (-(1/2) * phi**2 + (1/4) * phi**4
          + lamda * u * phi * (1 - (2/3) * phi**2 + (1/5) * phi**4))
free_energy = ((1/2) * W**2 * phi.grad.mag**2 + f_chem).cellVolumeAverage * nmx.sum(mesh.cellVolumes)
solid_fraction = ((phi + 1) / 2).cellVolumeAverage

In [23]:
class TipVariable(fp.Variable):
    def __init__(self, var):
        fp.Variable.__init__(self)
        self.var = self._requires(var)

    def _calcValue(self):
        spl = CubicSpline(self.var.mesh.x.value, self.var.value)
        sol = root_scalar(spl, bracket=(0, params["Lx"]))
        if sol.converged:
            return sol.root
        else:
            raise Exception

tip_position = TipVariable(phi)

In [24]:
faceNorm = phi.faceGrad / (phi.faceGrad.mag + (phi.faceGrad.mag == 0.) * 1.)
faceNorm.name = r"$\hat{n}$"
curvature = faceNorm.divergence
curvature.name = r"$\kappa$"

In [25]:
radius = 1. / curvature
radius.name = r"$\rho$"

## Initialize viewers

In [26]:
if params["view"]:
    phiviewer = fp.Viewer(vars=phi, datamin=-1, datamax=1)

In [27]:
# radviewer = fp.Viewer(vars=radius *(1 + phi) * (1-phi), datamin=-20, datamax=0)

In [28]:
# radviewer.plot()

In [29]:
# fp.tools.dump.write(mesh, "mesh.gz")

## Initialize fields

In [30]:
r = mesh.cellCenters.mag

In [35]:
t = 0.
phi.value = -1
phi.setValue(+1, where=r <= params["r0"])
u.value = -Delta

## Initialize output file

In [32]:
with open(output, "w") as f:
    f.write(f"t,free_energy,solid_fraction,tip_position\n")
    f.write(f"{t},{free_energy},{solid_fraction},{tip_position}\n")

## Solve in time

In [36]:
while t < params["t_max"]:
    phi.updateOld()
    u.updateOld()
    for sweep in range(3):
        res = eq.sweep(dt=params["dt"])
    t += params["dt"]
    if params["view"]:
        phiviewer.plot()
    with open(output, "a") as f:
        f.write(f"{t},{free_energy},{solid_fraction},{tip_position}\n")

t,free_energy,solid_fraction,tip_position
0.0,191.21537540893243,0.08342022940563103,8.008342022940564
0.5,177.10089483176802,0.09020475086027743,8.63804669296737
1.0,174.76265813362673,0.09949290758679095,9.508036375340891
1.5,172.66934374819684,0.10575578341631635,10.17150885751251
2.0,171.11596820073405,0.11098501388073197,10.698848517126255
2.5,169.82452850277153,0.11549017508326344,11.13993611350211
3.0,168.67801297552947,0.11945389588955689,11.521837726380905
3.5,167.6198594190266,0.1229969646527233,11.860564397113187
4.0,166.62247886212654,0.12620732131058657,12.166246316250906
4.5,165.6696693828246,0.12914745699905283,12.445666534528327
5.0,164.7513499721763,0.1318635515250279,12.703599895762967
5.5,163.86086086757575,0.13439062065845733,12.943548987566428
6.0,162.9931225686384,0.1367563155590137,13.167910486125932
6.5,162.14507507335202,0.1389807690230719,13.379089772955126
7.0,161.3144509199179,0.14108202693270935,13.57874569479177
7.5,160.4994984095107,0.14307466412825165,13